In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [2]:
anon = pd.read_csv('../Data_Anom/Lbitrik_542', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [3]:
print(anon)

                 id                date  latitude  longitude  week
0         36cb1f4f9 2015-03-08 22:00:00  4.750442  45.995189    10
1         282d2f323 2015-04-26 02:00:00  4.862690  45.749858    17
2         dd7b5e405 2015-04-28 23:00:00  4.859238  45.752762    18
3         e0331703a 2015-03-13 22:00:00  4.890035  45.758013    11
5         fcc8d31a4 2015-03-06 06:00:00  4.860258  45.752000    10
...             ...                 ...       ...        ...   ...
34551843  0215aaedc 2015-04-06 06:00:00  4.848578  45.778775    15
34551844  22e84f80f 2015-03-24 10:00:00  4.873977  45.794018    13
34551846  b1ff3a10c 2015-04-13 05:00:00  4.857891  45.784202    16
34551847  18bc28b8e 2015-03-19 02:00:00  4.856465  45.785643    12
34551848  c82de4323 2015-04-21 10:00:00  4.875759  45.790618    17

[29971332 rows x 5 columns]


In [4]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [5]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['0043e75c0' '0085d4660' '01b819574' '02054c712' '0215aaedc' '027075f9a'
 '032aa7a3e' '035bcfef1' '036a14343' '041665204' '0422eabb3' '059911b37'
 '05bbca085' '05dc5dc95' '060954726' '0649a5579' '069e0c834' '069ee7b23'
 '06af8cdc0' '083a900fa' '08a906129' '0913491cb' '0a25ed610' '0ad2d08fc'
 '0c3752c84' '0c81d307d' '0ca781f11' '0cf41591f' '0dea2dcd9' '0dfc289b1'
 '0e4b34713' '0e724988a' '0ea7353e6' '0f2e13a1e' '0fda45e6c' '10615d169'
 '11eefe717' '11fa2b2db' '1316681b4' '147d156ad' '14c1d21ae' '14dcfbeaf'
 '158339f64' '1896f4ffa' '18bc28b8e' '192872140' '198a4b33c' '1a069f959'
 '1a18c681c' '1a1c444f2' '1a5dbca36' '1a7277417' '1b29385a1' '1b5ce7817'
 '1c8c21851' '1cbbc1c82' '1cf6549e7' '1d700338e' '1d7a25e27

In [6]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [23]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)) ,1/2)
            
            list_distance.append(np.power(distance, 5)*delta_ratio*delta_hour)
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 1000000000
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

413
{'1': {'2015-10': ['d6a85d495'], '2015-11': ['f4322c952'], '2015-12': ['af117ee88'], '2015-13': ['75d68a074'], '2015-14': ['713805069'], '2015-15': ['bd5a21a7c'], '2015-16': ['4385f9e2e'], '2015-17': ['4a5de20a4'], '2015-18': ['035bcfef1'], '2015-19': ['ee9e6848b'], '2015-20': ['6e2f43d06']}, '2': {'2015-10': ['901553113'], '2015-11': ['90fb73c36'], '2015-12': ['89522f120'], '2015-13': ['0e4b34713'], '2015-14': ['398ad6d21'], '2015-15': ['1316681b4'], '2015-16': ['01b819574'], '2015-17': ['9e439f88d'], '2015-18': ['97a70f1bc'], '2015-19': ['3d8e5c61d'], '2015-20': ['946625dbc']}, '107': {'2015-10': ['b73650df0'], '2015-11': ['2458d3a9f'], '2015-12': ['46da38734']}, '98': {'2015-10': ['aadcb9c89'], '2015-11': ['ffca98903']}, '21': {'2015-10': ['f795e8ca6'], '2015-11': ['8a96fcb26']}, '5': {'2015-10': ['8ed5ab15e'], '2015-11': ['fdb5ab41a']}, '9': {'2015-10': ['9d4abd6ed'], '2015-11': ['e84210ef1'], '2015-12': ['af117ee88'], '2015-13': ['38b49fa99'], '2015-14': ['eb676c434'], '2015-1

In [24]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "d6a85d495"
        ],
        "2015-11": [
            "f4322c952"
        ],
        "2015-12": [
            "af117ee88"
        ],
        "2015-13": [
            "75d68a074"
        ],
        "2015-14": [
            "713805069"
        ],
        "2015-15": [
            "bd5a21a7c"
        ],
        "2015-16": [
            "4385f9e2e"
        ],
        "2015-17": [
            "4a5de20a4"
        ],
        "2015-18": [
            "035bcfef1"
        ],
        "2015-19": [
            "ee9e6848b"
        ],
        "2015-20": [
            "6e2f43d06"
        ]
    },
    "2": {
        "2015-10": [
            "901553113"
        ],
        "2015-11": [
            "90fb73c36"
        ],
        "2015-12": [
            "89522f120"
        ],
        "2015-13": [
            "0e4b34713"
        ],
        "2015-14": [
            "398ad6d21"
        ],
        "2015-15": [
            "1316681b4"
        ],
        "2015-16":